<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/price/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import joblib
import pandas as pd

def fillna_empty(x):
    return x.fillna('')

def fillna_str(x):
    return x.fillna('').astype(str)

# 2.1 โหลดโมเดลจากไฟล์ .pkl
model = joblib.load("/content/drive/MyDrive/AIEngineer/price/demo/random_forest_model.pkl")

In [33]:
model

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['OcmPatTyp',
                                                   'NurseStation']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['IncomeCode']),
                                                 ('text_CC',
                                                  Pipeline(steps=[('fillna',
                                                                   FunctionTransformer(func=<function fillna_empty at 0x784e476055...
                                                                   TfidfVectorizer(max_df=0.7,
                                                                                   max_features=500,
                                                                                   min_df=5,
                                                                                   ngram_range=(1,
                                                                                                2)))]),
                                                  'docname'),
                                                 ('text_ICD10',
                                                  Pipeline(steps=[('fillna',
                                                                   FunctionTransformer(func=<function fillna_empty at 0x784e47605580>)),
                                                                  ('tfidf',
                                                                   TfidfVectorizer(max_df=0.7,
                                                                                   max_features=500,
                                                                                   min_df=5,
                                                                                   ngram_range=(1,
                                                                                                2)))]),
                                                  'ICD10')])),
                ('regressor',
                 RandomForestRegressor(n_jobs=-1, random_state=42))])

In [59]:
import pandas as pd

income_type = pd.read_csv('/content/drive/MyDrive/AIEngineer/price/RYH_PatIncom_202412.csv', encoding='cp874')
summary = pd.read_csv('/content/drive/MyDrive/AIEngineer/price/demo/summary.csv')
X = pd.read_csv('/content/drive/MyDrive/AIEngineer/price/demo/X_test.csv')
Y = pd.read_csv('/content/drive/MyDrive/AIEngineer/price/demo/Y_test.csv')
df = pd.read_csv('/content/drive/MyDrive/AIEngineer/price/RYH_DrNote_202412.csv', encoding='cp874')

In [60]:
common_ids = sorted(list(set(df['ID']) & set(income_type['ID'])))
def get_demo_data_by_id(selected_id):
    row = df[df['ID'] == selected_id].iloc[0]
    # Return fields in same order as your gradio inputs!
    return (
        row['OcmPatTyp'],
        row['NurseStation'],
        row['CC'],
        row['Pi'],
        row['Diag'],
        row['Plan'],
        row['Inv'],
        row['PEtext'],
        row['ICD10'],
        row['PhyDtrCod'],
        row['docname']
    )

In [61]:
import os
import os
import json
import requests
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import re
import ast
import time

# ===== LLM API CONFIGURATION =====
LLM_API_URL = "https://ai-api.manageai.co.th/llm-model-02/generate"
LLM_AUTH = ("manageai2024", "ManageAI@2024")
model_param = {
    "best_of": 1,
    "frequency_penalty": 1.1,
    "max_new_tokens": 600,
    "repetition_penalty": 1.1,
    "temperature": 0.1,
    "top_k": 10,
    "top_n_tokens": 5,
    "top_p": 0.95,
    "stop": ["assistant"]
}

# === Call LLM
def generate_response(prompt, retries=1, delay=1):
    headers = {"Content-Type": "application/json"}
    data = json.dumps({"inputs": prompt, "parameters": model_param})

    def call_api():
        try:
            response = requests.post(
                LLM_API_URL, data=data, headers=headers,
                auth=LLM_AUTH, timeout=60
            )
            if response.status_code == 200:
                # Adjust here if your API response is different
                result = response.json().get("generated_text", "").strip()
                if result:
                    return result
                else:
                    return None
            else:
                return f"ERROR: Status {response.status_code}"
        except requests.exceptions.Timeout:
            print("⚠️ Timeout... will retry if retries remain.")
            return None
        except Exception as e:
            return f"ERROR: {e}"

    result = call_api()
    tries = 0

    while (not result or (isinstance(result, str) and result.startswith("ERROR"))) and tries < retries:
        tries += 1
        print(f"Retry {tries}/{retries}...")
        time.sleep(delay)
        result = call_api()
    return result if result else "ERROR: Maximum retries exceeded"


# === Extract code
def extract_code(icd10_raw):
    if pd.isna(icd10_raw):
        return None
    if icd10_raw == '[9999] Unspecified':
        return 'R69'
    if icd10_raw == '[่่J06.9] Acute upper respiratory infection, unspecified':
        return 'J069'
    match = re.search(r'\[([A-Z]\d{2,3}(?:\.\d{1,3})?)\]', str(icd10_raw))
    if match:
        code = match.group(1).replace('.', '')
        return code[:5]
    return None

## **prompt**

In [96]:
def clean_field(val):
    return "-" if pd.isna(val) or val == "" else str(val)

def build_income_prompt(row, summary, income_type, topn=5):
    code = extract_code(row['ICD10'])
    phy = str(row['PhyDtrCod']).strip()
    nurse = str(row['NurseStation']).strip()
    pattype = str(row['OcmPatTyp']).strip()

    # Now filter by Code, PhyDtrCod, and NurseStation
    df_ref = summary[
        (summary['Code'] == code) &
        (summary['PhyDtrCod'] == phy) &
        (summary['NurseStation'] == nurse) &
        (summary['OcmPatTyp'] == pattype)
    ]

    income_type = income_type[['IncomeCode','IncomeCode.1']].value_counts().reset_index().drop(columns=['count'])
    top_income = df_ref.sort_values('count', ascending=False)[['IncomeCode', 'IncomeCode.1']]
    top_income_unique = top_income.drop_duplicates(subset=['IncomeCode', 'IncomeCode.1'])
    choices = [f"{r['IncomeCode']} : {r['IncomeCode.1']}" for _, r in top_income_unique.iterrows()]

    # choices = []
    # for _, r in top_income.iterrows():
    #     choices.append(f"{r['IncomeCode']} : {r['IncomeCode.1']}")

    prompt = f"""
You are a medical billing specialist. For the following patient record, predict the lists of IncomeCode and IncomeCode.1 for this visit.

Patient record:
ประเภทผู้ป่วย: {clean_field(row.get('OcmPatTyp'))}
แผนกที่รับการตรวจ: {clean_field(row.get('NurseStation'))}
อาการสำคัญ (Chief Complaint): {clean_field(row.get('CC'))}
ประวัติอาการปัจจุบัน (Present Illness): {clean_field(row.get('Pi'))}
วินิจฉัยโรค (Diagnosis): {clean_field(row.get('Diag'))}
แผนการรักษา (Treatment Plan): {clean_field(row.get('Plan'))}
การตรวจทางห้องปฏิบัติการ (Investigation): {clean_field(row.get('Inv'))}
ผลการตรวจร่างกาย (Physical Examination): {clean_field(row.get('PEtext'))}
รหัสวินิจฉัยโรคตามมาตรฐาน ICD-10: {clean_field(row.get('ICD10'))}
รหัสแพทย์ที่ตรวจ: {clean_field(row.get('PhyDtrCod'))}
ชื่อแพทย์ที่ตรวจ: {clean_field(row.get('docname'))}

Reference income code options for this ICD10/doctor/NurseStation/OcmPatTyp:
{chr(10).join(choices) if choices else '[No context. Use medical context.]'}

**Very important rules:**
- Use only codes from the reference list for this patient record. Never output a code that does not appear in the reference list. If the reference list is empty, you may use medical knowledge as context.
- Please include **ALL codes from the reference list that are possibly applicable** based on any information in the patient record (Plan, CC, Pi, ICD10, etc.). It is better to include all reasonable codes than to miss any. Do not omit a code if there is any reasonable evidence it applies.
- Do **NOT** include a code just because it appears frequently or seems “default.” Only include it if the clinical context supports it.
- **Do not hallucinate or invent codes.**

Special rule:
- Use both the clinical context (Plan, CC, Pi) and diagnosis (ICD10) to determine which codes should be included.
- If OcmPatTyp is 'O' (OPD):
    1. **Medicine or Vaccine:**
        - Include 6 in the [IncomeCode list] and '1.1.1 (3) ค่ายาผู้ป่วยนอก' in the [IncomeCode.1 list] in the following cases:
            - If there are '6 : 1.1.1 (3) ค่ายาผู้ป่วยนอก' in the Reference income code options.
            - **Or, If any of these columns—Plan, CC, or Pi—explicitly mention a keyword indicating the patient will be given medicine or a vaccine (such as "ยา", "จ่ายยา", "รับยา", "สั่งยา", "ให้ยา", "ยาหมด", "med", "prescribe", "medication", "vaccine", "ฉีดวัคซีน", "วัคซีน", "dispense", etc. in Thai or English).
    2. **Wound care, use of medical equipment, or procedures:**
        - If any of these columns—Plan, CC, or Pi—mention wound care, the use of medical supplies/equipment, or any surgical/procedural action (such as "ทำแผล", "แผล", "ผ่าตัด", "หัตถการ", "wound", "dressing", "procedure", "operation", "instrument", "suture", "dressing", etc.), then **include some or all of the following codes in the lists as appropriate:**
            - 13 ('1.1.2 (1) เวชภัณฑ์ 1 ค่าเวชภัณฑ์สิ้นเปลืองทางการแพทย์')
            - 44 ('1.1.7 (1) ค่าอุปกรณ์ของใช้และเครื่องมือทางการแพทย์ทั่วไป')
            - 43 ('1.1.7 ค่าอุปกรณ์ของใช้และเครื่องมือทางการแพทย์')
            - 122 ('1.2.2 ค่าทำศัลยกรรมและหัตถการต่างๆ ของผู้ประกอบวิชาชีพ')
        - **Additionally, if the ICD10 diagnosis code indicates a wound, injury, trauma, or procedure** (for example: codes starting with 'S' or 'T', or relevant Z-codes), you must also include the appropriate codes for wound care, medical supplies, or equipment—even if the Plan, CC, or Pi columns do not mention them explicitly.
- Keep ONLY incomecodes that are appropriate for this patient.

**Negative examples:**
- Do NOT include equipment/procedure codes unless supported by text or ICD10.

**Output rules:**
- Reply with EXACTLY two lists of answer, no explanation, no extra text.

For example:
[6, 92, 104]
['1.1.1 (3) ค่ายาผู้ป่วยนอก', '1.1.14 (1) ค่าบริการชุดเหมาจ่าย', '1.2.1 ค่าตรวจรักษาทั่วไปของผู้ประกอบวิชาชีพ']

Now, predict the lists for this record.
""".strip()
    # print(prompt)
    return prompt

import re
import ast

# === Predict for one row
def predict_income(row, summary, income_type):
    prompt = build_income_prompt(row, summary, income_type)
    return generate_response(prompt)

def split_predicted_income(val):
    if not isinstance(val, str):
        return None, None
    lists = []
    try:
        # ดึง list ทั้งหมดที่อยู่ในข้อความ
        found = re.findall(r'\[.*?\]', val, re.DOTALL)
        for l in found:
            # ลองแปลงเป็น list จริง
            try:
                l_eval = ast.literal_eval(l)
                lists.append(l_eval)
            except:
                continue

        # หา list ที่ “เป็นตัวเลขล้วน”
        income_codes = None
        income_types = None
        for l in lists:
            # ถ้าเป็น list ของตัวเลข (int/float)
            if isinstance(l, list) and len(l) > 0 and all(isinstance(x, (int, float)) for x in l):
                income_codes = l
            # ถ้าเป็น list ของ string (อย่างน้อย 1 ตัว)
            elif isinstance(l, list) and len(l) > 0 and all(isinstance(x, str) for x in l):
                income_types = l

        # ถ้าหาเจอทั้งคู่
        if income_codes is not None and income_types is not None:
            return income_codes, income_types

        # fallback: ถ้าไม่ได้ ให้ดูแบบเก่า
        if len(lists) >= 2:
            return lists[0], lists[1]
    except Exception as e:
        pass
    return None, None


# === Batch predict and parse (Full pipeline)
def predict_income_parallel(df, summary, income_type, max_workers=10):
    results = [None] * len(df)
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(predict_income, row, summary, income_type): i
            for i, (_, row) in enumerate(df.iterrows())
        }
        for future in tqdm(futures, total=len(futures), desc="Predicting IncomeCode"):
            i = futures[future]
            try:
                results[i] = future.result()
            except Exception as e:
                results[i] = f"ERROR: {e}"
    df = df.copy()
    df['predicted_income'] = results
    # Split and expand
    pred_split = df['predicted_income'].apply(lambda x: pd.Series(split_predicted_income(x)))
    df['IncomeCode_pred'] = pred_split[0]
    df['IncomeCode.1_pred'] = pred_split[1]
    return df

# df_predicted = predict_income_parallel(X_test, summary, income_type)


In [97]:
import gradio as gr
import pandas as pd

In [98]:
def gradio_predict_single(
    OcmPatTyp, NurseStation, CC, Pi, Diag, Plan, Inv, PEtext, ICD10, PhyDtrCod, docname
):
    row = {
        'OcmPatTyp': OcmPatTyp, 'NurseStation': NurseStation,
        'CC': CC, 'Pi': Pi, 'Diag': Diag, 'Plan': Plan, 'Inv': Inv,
        'PEtext': PEtext, 'ICD10': ICD10, 'PhyDtrCod': PhyDtrCod, 'docname': docname
    }
    result = predict_income(pd.Series(row), summary, income_type)
    codes, names = split_predicted_income(result)
    # If None, set to empty
    if codes is None: codes = []
    if names is None: names = []
    length = max(len(codes), len(names))
    codes = codes if len(codes) == length else (codes + [""] * (length - len(codes)))
    names = names if len(names) == length else (names + [""] * (length - len(names)))

    # For each IncomeCode, predict Amt
    rows = []
    for code, code1 in zip(codes, names):
        feature_row = {
            'OcmPatTyp': OcmPatTyp,
            'NurseStation': NurseStation,
            'CC': CC,
            'Pi': Pi,
            'Diag': Diag,
            'Plan': Plan,
            'Inv': Inv,
            'PEtext': PEtext,
            'ICD10': ICD10,
            'docname': docname,
            'IncomeCode': code
        }
        # Create DataFrame with single row for the model
        X = pd.DataFrame([feature_row])
        amt_pred = None
        try:
            amt_pred = float(model.predict(X)[0])
        except Exception as e:
            amt_pred = "ERROR"
        rows.append([code, code1, amt_pred])

    # Return as DataFrame with new column
    return pd.DataFrame(rows, columns=["IncomeCode", "IncomeCode.1", "Predicted Amt"])

def gradio_predict_batch(file):
    import pandas as pd
    # Read file
    df = pd.read_csv(file.name if hasattr(file, 'name') else file)

    results = []
    for idx, row in df.iterrows():
        row_dict = row.to_dict()
        # Call LLM (prompt-based) prediction
        llm_result = predict_income(pd.Series(row_dict), summary, income_type)
        codes, names = split_predicted_income(llm_result)
        if codes is None: codes = []
        if names is None: names = []
        length = max(len(codes), len(names))
        codes = codes if len(codes) == length else (codes + [""] * (length - len(codes)))
        names = names if len(names) == length else (names + [""] * (length - len(names)))

        for code, code1 in zip(codes, names):
            feature_row = {
                'OcmPatTyp': row_dict.get('OcmPatTyp', ''),
                'NurseStation': row_dict.get('NurseStation', ''),
                'CC': row_dict.get('CC', ''),
                'Pi': row_dict.get('Pi', ''),
                'Diag': row_dict.get('Diag', ''),
                'Plan': row_dict.get('Plan', ''),
                'Inv': row_dict.get('Inv', ''),
                'PEtext': row_dict.get('PEtext', ''),
                'ICD10': row_dict.get('ICD10', ''),
                'docname': row_dict.get('docname', ''),
                'IncomeCode': code
            }
            # Predict Amt for each code
            X = pd.DataFrame([feature_row])
            try:
                amt_pred = float(model.predict(X)[0])
            except Exception as e:
                amt_pred = "ERROR"
            results.append({
                'ID': row_dict.get('ID', idx),  # Keep original ID for traceability
                'IncomeCode': code,
                'IncomeCode.1': code1,
                'Predicted Amt': amt_pred
            })

    return pd.DataFrame(results, columns=["ID", "IncomeCode", "IncomeCode.1", "Predicted Amt"])


## **eval**

In [99]:
import numpy as np

def evaluate_prediction(selected_id):
    # -- Get patient record --
    row = df[df['ID'] == selected_id].iloc[0]
    input_args = (
        row['OcmPatTyp'], row['NurseStation'], row['CC'], row['Pi'], row['Diag'],
        row['Plan'], row['Inv'], row['PEtext'], row['ICD10'], row['PhyDtrCod'], row['docname']
    )
    # -- Predict with your pipeline --
    pred_df = gradio_predict_single(*input_args)  # This returns a DataFrame

    # -- Get ground truth --
    gt = income_type[income_type['ID'] == selected_id]
    gt = gt.rename(columns={"Amt": "True Amt"})  # For clarity

    # -- Merge predicted with GT on IncomeCode and IncomeCode.1 --
    merged = pred_df.merge(
        gt,
        left_on=["IncomeCode", "IncomeCode.1"],
        right_on=["IncomeCode", "IncomeCode.1"],
        how='outer',
        suffixes=('_pred', '_gt'),
        indicator=True
    )

    # -- Add evaluation columns --
    def match_status(row):
        if row['_merge'] == 'both':
            # Compare Amt
            try:
                pred_amt = float(row['Predicted Amt'])
                true_amt = float(row['True Amt'])
                if np.isclose(pred_amt, true_amt, atol=1e-2):  # tolerance for float
                    return "✅ ทำนายถูกต้อง"
                else:
                    diff = true_amt - pred_amt
                    return f"❎ คลาดเคลื่อน {true_amt:.2f} - {pred_amt:.2f} = {diff:+.2f}"
            except:
                return "Amt Error"
        elif row['_merge'] == 'left_only':
            return "❓ ทำนาย incomecode เกิน"
        elif row['_merge'] == 'right_only':
            return "❌ ไม่พบในทำนาย"
        return "Unknown"


    merged['Status'] = merged.apply(match_status, axis=1)

    # -- Arrange columns for display --
    show_cols = [
        "IncomeCode", "IncomeCode.1", "Predicted Amt", "True Amt", "Status"
    ]
    return merged[show_cols]


In [100]:
def evaluate_income_code_predictions(df, pred_col='IncomeCode_pred', true_col='IncomeCode'):
    import ast
    def ensure_list(val):
        if isinstance(val, str):
            return ast.literal_eval(val)
        return val
    pred_lists = df[pred_col].apply(ensure_list)
    true_lists = df[true_col].apply(ensure_list)
    exact_matches = []
    precision_list = []
    recall_list = []
    f1_list = []
    for pred, true in zip(pred_lists, true_lists):
        pred_set = set(pred)
        true_set = set(true)
        exact = pred_set == true_set
        exact_matches.append(exact)
        tp = len(pred_set & true_set)
        fp = len(pred_set - true_set)
        fn = len(true_set - pred_set)
        precision = tp / (tp + fp) if tp + fp else 0
        recall = tp / (tp + fn) if tp + fn else 0
        f1 = 2 * precision * recall / (precision + recall) if precision + recall else 0
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)
    df['exact_match'] = exact_matches
    df['precision'] = precision_list
    df['recall'] = recall_list
    df['f1'] = f1_list
    total = len(df)
    accuracy = sum(exact_matches) / total
    mean_precision = sum(precision_list) / total
    mean_recall = sum(recall_list) / total
    mean_f1 = sum(f1_list) / total
    # Markdown summary for Gradio
    summary = (
        f"**Exact match accuracy:** {accuracy:.2%}  \n"
        f"**Mean Precision:** {mean_precision:.2%}  \n"
        f"**Mean Recall:** {mean_recall:.2%}  \n"
        f"**Mean F1-score:** {mean_f1:.2%}"
    )
    return summary, df[[pred_col, true_col, 'exact_match', 'precision', 'recall', 'f1']]

In [101]:
def evaluate_income_code_single(pred_list, true_list):
    data = {
        'IncomeCode_pred': [pred_list],
        'IncomeCode': [true_list]
    }
    df = pd.DataFrame(data)
    summary, eval_df = evaluate_income_code_predictions(df)  # <-- unpack!
    precision = eval_df['precision'].iloc[0]
    recall = eval_df['recall'].iloc[0]
    f1 = eval_df['f1'].iloc[0]
    return f"{precision:.2%}", f"{recall:.2%}", f"{f1:.2%}"


In [105]:
def predict_and_evaluate(
    OcmPatTyp, NurseStation, CC, Pi, Diag, Plan, Inv, PEtext, ICD10, PhyDtrCod, docname, id_dropdown_value
):
    # Run prediction
    pred_table = gradio_predict_single(
        OcmPatTyp, NurseStation, CC, Pi, Diag, Plan, Inv, PEtext, ICD10, PhyDtrCod, docname
    )
    # Get predicted IncomeCode list
    pred_list = list(pred_table['IncomeCode'])
    if id_dropdown_value:
        gt_list = list(income_type[income_type['ID']==id_dropdown_value]['IncomeCode'])
        precision, recall, f1 = evaluate_income_code_single(pred_list, gt_list)
    else:
        precision, recall, f1 = "", "", ""
    # Evaluation table as before
    eval_table = None
    try:
        if id_dropdown_value:
            eval_table = evaluate_prediction(id_dropdown_value)
        else:
            eval_table = pred_table.assign(**{'True Amt': '', 'Status': ''})[
                ["IncomeCode", "IncomeCode.1", "Predicted Amt", "True Amt", "Status"]
            ]
    except Exception as e:
        eval_table = pred_table.assign(**{'True Amt': '', 'Status': f'Error: {e}'})[
            ["IncomeCode", "IncomeCode.1", "Predicted Amt", "True Amt", "Status"]
        ]
    # --- Sort eval_table by IncomeCode before return ---
    eval_table = eval_table.sort_values(by="IncomeCode").reset_index(drop=True)
    return pred_table, eval_table, precision, recall, f1


## **demo**

In [106]:
import gradio as gr

with gr.Blocks(title="Medical Billing Predictor") as demo:
    gr.Markdown("# Medical Billing Predictor")
    gr.Markdown("This demo uses LLM and trained model to predict billing codes and each billing amount from patient data.")

    # --- SINGLE PATIENT TAB ---
    with gr.Tab("Single Patient Prediction"):
        with gr.Column():
            id_dropdown = gr.Dropdown(choices=common_ids, label="Select Sample Patient ID")
        with gr.Row():
            inp_type = gr.Textbox(label="Patient Type (O/I)", placeholder="O")
            inp_nurse = gr.Textbox(label="Nurse Station", placeholder="OPD ทั่วไป")
        with gr.Row():
            inp_cc = gr.Textbox(label="Chief Complaint", placeholder="นัดติดตามอาการ")
            inp_pi = gr.Textbox(label="Present Illness", placeholder="F/U headache\nปวดทุกสัปดาห์ สัปดาห์ละ 1 ครั้ง pain 7/10\nexcercise: ok\nsleep: ok")
            inp_diag = gr.Textbox(label="Diagnosis", placeholder="ปวดศีรษะ ไม่ระบุรายละเอียด(Headache, unspecified)")
        with gr.Row():
            inp_plan = gr.Textbox(label="Treatment Plan", placeholder="-")
            inp_inv = gr.Textbox(label="Investigation", placeholder="-")
        with gr.Row():
            inp_pe = gr.Textbox(label="Physical Examination", placeholder="good consciousness\npupil 2.5 mmB")
            inp_icd10 = gr.Textbox(label="ICD10", placeholder="[S33.5] Sprain and strain of lumbar spine")
        with gr.Row():
            inp_phycode = gr.Textbox(label="Physician Code", placeholder="1234")
            inp_docname = gr.Textbox(label="Physician Name", placeholder="นพ. สมชาย")
        button = gr.Button("Predict")

        with gr.Row():
            result_table = gr.Dataframe(
                headers=["IncomeCode", "IncomeCode.1", "Predicted Amt"],
                label="Predicted Income Codes and Amt",
                interactive=False,
                wrap=True
            )

        with gr.Row():
            single_precision = gr.Textbox(label="Precision", interactive=False)
            single_recall = gr.Textbox(label="Recall", interactive=False)
            single_f1 = gr.Textbox(label="F1-score", interactive=False)

        with gr.Row():
            eval_table = gr.Dataframe(
                headers=["IncomeCode", "IncomeCode.1", "Predicted Amt", "True Amt", "Status"],
                label="Patient Evaluation Detail",
                interactive=False
            )

        # Remove the button!
        id_dropdown.change(
            get_demo_data_by_id,
            inputs=id_dropdown,
            outputs=[
                inp_type, inp_nurse, inp_cc, inp_pi, inp_diag, inp_plan,
                inp_inv, inp_pe, inp_icd10, inp_phycode, inp_docname
            ]
        )

        button.click(
            predict_and_evaluate,
            inputs=[
                inp_type, inp_nurse, inp_cc, inp_pi, inp_diag, inp_plan,
                inp_inv, inp_pe, inp_icd10, inp_phycode, inp_docname,
                id_dropdown
            ],
            outputs=[result_table, eval_table, single_precision, single_recall, single_f1]
        )

    # # --- BATCH PATIENT TAB ---
    # with gr.Tab("Batch CSV Prediction"):
    #     file_input = gr.File(label="Upload patient CSV")
    #     batch_table = gr.Dataframe(
    #         headers=["ID", "IncomeCode", "IncomeCode.1", "Predicted Amt"],
    #         label="Prediction Results",
    #         interactive=False,
    #         wrap=True
    #     )
    #     gr.Button("Predict Batch").click(
    #         gradio_predict_batch,
    #         inputs=file_input,
    #         outputs=batch_table
    #     )

demo.launch(debug=True)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://86854d5d73aef6894b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://86854d5d73aef6894b.gradio.live
